In [2]:
import h2o

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


--------------------------  -----------------------------
H2O_cluster_uptime:         3 hours 28 mins
H2O_cluster_timezone:       Asia/Kolkata
H2O_data_parsing_timezone:  UTC
H2O_cluster_version:        3.46.0.1
H2O_cluster_version_age:    29 days
H2O_cluster_name:           sysadm
H2O_cluster_total_nodes:    2
H2O_cluster_free_memory:    3.639 Gb
H2O_cluster_total_cores:    8
H2O_cluster_allowed_cores:  8
H2O_cluster_status:         locked, healthy
H2O_connection_url:         http://localhost:54321
H2O_connection_proxy:       {"http": null, "https": null}
H2O_internal_security:      False
Python_version:             3.10.12 final
--------------------------  -----------------------------

In [5]:
h2o.cluster_status()

/tmp/ipykernel_59538/2774355338.py:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().show_status(True)``.
  h2o.cluster_status()


--------------------------  -----------------------------
H2O_cluster_uptime:         3 hours 28 mins
H2O_cluster_timezone:       Asia/Kolkata
H2O_data_parsing_timezone:  UTC
H2O_cluster_version:        3.46.0.1
H2O_cluster_version_age:    29 days
H2O_cluster_name:           sysadm
H2O_cluster_total_nodes:    2
H2O_cluster_free_memory:    3.639 Gb
H2O_cluster_total_cores:    8
H2O_cluster_allowed_cores:  8
H2O_cluster_status:         locked, healthy
H2O_connection_url:         http://localhost:54321
H2O_connection_proxy:       {"http": null, "https": null}
H2O_internal_security:      False
Python_version:             3.10.12 final
--------------------------  -----------------------------

Nodes info:     Node 1                             Node 2
--------------  ---------------------------------  ---------------------------------
h2o             172.20.241.21/172.20.241.21:54321  172.20.241.39/172.20.241.39:54321
healthy         True                               True
last_ping       1712869432894                      1712869432965
num_cpus        4                                  4
sys_load        4.8789062                          0.1796875
mem_value_size  2398401                            1943443
free_mem        2145085247                         1762358576
pojo_mem        0                                  144630589
swap_mem        0                                  0
free_disk       64883785728                        72583479296
max_disk        97826897920                        97826897920
pid             16990                              3541
num_keys        507                                282
tcps_active     2                                  2
open_fds        34                                 53
rpcs_active     0                                  0

# File Transfer

In [7]:
loans = h2o.import_file("https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-at-Scale-with-H2O/main/chapt2/loans-lite.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [13]:
h2o.export_file(loans, "loans.csv", force=True)

Export File progress: |██████████████████████████████████████████████████████████| (done) 100%


In [14]:
data2=h2o.upload_file("loans.csv",destination_frame="loans")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


# Distributive Random Forest

In [17]:
# import h2o
from h2o.estimators import H2ORandomForestEstimator
# h2o.init()

# Import the cars dataset into H2O:
cars = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/junit/cars_20mpg.csv")

# Set the predictors and response;
# set the response as a factor:
cars["economy_20mpg"] = cars["economy_20mpg"].asfactor()
predictors = ["displacement","power","weight","acceleration","year"]
response = "economy_20mpg"

# Split the dataset into a train and valid set:
train, valid = cars.split_frame(ratios=[.8], seed=1234)

# Build and train the model:
cars_drf = H2ORandomForestEstimator(ntrees=10,
                                    max_depth=5,
                                    min_rows=10,
                                    calibrate_model=True,
                                    calibration_frame=valid,
                                    binomial_double_trees=True)
cars_drf.train(x=predictors,
               y=response,
               training_frame=train,
               validation_frame=valid)

# Eval performance:
perf = cars_drf.model_performance()

# Generate predictions on a validation set (if necessary):
pred = cars_drf.predict(valid)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [18]:
pred

predict         p0         p1    cal_p0     cal_p1
---------  ---------  ---------  --------  ---------
        0  0.616231   0.383769   0.864319  0.135681
        0  0.753621   0.246379   0.944036  0.0559637
        0  0.786687   0.213313   0.955205  0.044795
        0  1          0          0.989766  0.0102336
        0  1          0          0.989766  0.0102336
        0  1          0          0.989766  0.0102336
        0  0.963636   0.0363636  0.986797  0.0132026
        0  0.945051   0.0549491  0.984966  0.0150337
        0  1          0          0.989766  0.0102336
        1  0.0895026  0.910497   0.13218   0.86782
[76 rows x 5 columns]

In [19]:
perf

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.06928619703096771
RMSE: 0.2632227137444026
LogLoss: 0.4216697750197926
Mean Per-Class Error: 0.079371921182266
AUC: 0.9588054187192119
AUCPR: 0.9717161439488978
Gini: 0.9176108374384238

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.625
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      111  9    0.075    (9.0/120.0)
1      17   186  0.0837   (17.0/203.0)
Total  128  195  0.0805   (26.0/323.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.625        0.934673  74
max f2                       0.102083     0.94722   127
max f0point5                 0.625        0.946083  74
max accuracy                 0.625        0.919505  74
max precision                0.958399     0.992126  14
max recall                   0            1         152
max specificity              1            0.991667  0
max absolute_mcc             0.625        0.831083  74
max min_per_class_accuracy   0.625        0.916256  74
max mean_per_class_accuracy  0.625        0.920628  74
max tns                      1            119       0
max fns                      1            99        0
max fps                      0            120       152
max tps                      0            203       152
max tnr                      1            0.991667  0
max fnr                      1            0.487685  0
max fpr                      0            1         152
max tpr                      0            1         152

Gains/Lift Table: Avg response rate: 62.77 %, avg score: 62.23 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.323077                    1                  1.57796   1.57796            0.990476         1           0.990476                    1                   0.509804        0.509804                   57.7965   57.7965            0.501539
2        0.406154                    0.941414           1.47513   1.55693            0.925926         0.972579    0.977273                    0.994391            0.122549        0.632353                   47.5127   55.693             0.60756
3        0.507692                    0.864029           1.44831   1.5352             0.909091         0.907633    0.963636                    0.977039            0.147059        0.779412                   44.8307   53.5205            0.729825
4        0.609231                    0.568941           1.35175   1.50463            0.848485         0.762353    0.944444                    0.941258            0.137255        0.916667                   35.1753   50.463             0.825758
5        0.713846                    0.2                0.46857   1.35279            0.294118         0.351961    0.849138                    0.854896            0.0490196       0.965686                   -53.143   35.2793            0.67643
6        0.812308                    0.0464822          0.199142  1.21296            0.125            0.110177    0.761364                    0.764627            0.0196078       0.985294                   -80.0858  21.2957            0.464633
7        1                           0                  0.078351  1                  0.0491803        0.00645636  0.627692                    0.622324            0.0147059       1                          -92.1649  0                  0